In [1]:
pip install fastapi


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-multipart


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



In [3]:
from threading import Thread

In [4]:
from fastapi import FastAPI, Request, Form
from fastapi.templating import Jinja2Templates
from fastapi.responses import RedirectResponse
from datetime import datetime, timedelta
from typing import List, Optional
import json, random, time
import glob

In [5]:
import logging

# 로그 설정
logging.basicConfig(
    filename="server.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    encoding="utf-8"
)
log = logging.getLogger(__name__)

In [6]:
from contextlib import asynccontextmanager

In [7]:
# status_info 전역 변수 선언
status_info = {}

# FAST API 시작 부분 - app start 대체
@asynccontextmanager
async def lifespan(app: FastAPI):
    try:
        with open("tickers.json", "r") as f:
            tickers_data = json.load(f)
    except FileNotFoundError:
        tickers_data = []

    for ex in tickers_data:
        #ex = entry["exchange"]
        for ticker in tickers_data.get(ex, []):
            print("📌 Exchange:", ex, "Ticker:", ticker)
        
        if ex not in status_info:
            status_info[ex] = {
                "status": "Running",
                "last_updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
    # 앱 시작 시
    log.info("✅ App started")
    log.info("✅ Starting status_updater thread")
            
    thread = Thread(target=status_updater, daemon=True)
    thread.start()

    yield  # startup 이후 본격 앱 실행

    # 앱 종료
    log.info("🔴 App shutting down...")

# FastAPI 앱 선언 시 lifespan 추가
app = FastAPI(lifespan=lifespan)

In [8]:
# 삭제가 안되는 default ticker 지정
default_tickers = {
    "Upbit": ["BTC/KRW"],
    "Korbit": ["BTC/KRW"],
    "Bithumb": ["BTC/KRW"],
    "Coinone": ["BTC/KRW"]
}


# 에러 핸들러 등록
@app.exception_handler(Exception)
async def global_exception_handler(request: Request, exc: Exception):
    log.error(f"❗ Error occurred: {exc}")
    # 예외 발생 시 자동으로 로그 남기고, 사용자에게는 500 응답
    return JSONResponse(status_code=500, content={"detail": "Internal Server Error"})

# 서버 상태 확인
def status_updater():
    """Background thread function to update exchange statuses and timestamps."""
    while True:
        for ex, info in list(status_info.items()):
            if info["status"] == "Running":
                # Update last-updated timestamp for running collectors
                info["last_updated"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            # Randomly simulate an error state (~10% chance each cycle)
            if random.random() < 0.1:
                info["status"] = "Error" if info["status"] == "Running" else "Running"
        time.sleep(5)  # wait 5 seconds before next update cycle

In [9]:
# KST Seoul 기준 D-1일
try:
    from zoneinfo import ZoneInfo
    KST = ZoneInfo("Asia/Seoul")
except Exception:
    from datetime import timezone, timedelta as td
    KST = timezone(td(hours=9))

def get_previous_day_str():
    """Return yesterday's date as YYYYMMDD in KST."""
    now_kst = datetime.now(tz=KST)
    yesterday = now_kst - timedelta(days=1)
    return yesterday.strftime("%Y%m%d")


In [10]:
templates = Jinja2Templates(directory="templates")

# 모든 html 파일은 templetes에 있음
# templates/index.html


@app.get("/")
async def main_page(request: Request):
    # Load tickers from JSON
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    # Sort exchanges and tickers for consistent display order (optional)
    sorted_tickers = {ex: sorted(tickers_data[ex]) for ex in sorted(tickers_data)}
    context = {
        "request": request,
        "tickers_data": sorted_tickers,
        "selected_tickers": [],        # no tickers selected by default
        "result_data": None,           # no results to display on initial load
        "date_str": get_previous_day_str(),  # previous day date for reference
        "default_tickers": default_tickers
    }
    return templates.TemplateResponse("index.html", context)

    
@app.post("/select")
async def show_orderbook_data(request: Request, tickers: Optional[List[str]] = Form(None)):
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    sorted_tickers = {ex: sorted(tickers_data[ex]) for ex in sorted(tickers_data)}

    if not tickers:
        return templates.TemplateResponse("index.html", {
            "request": request,
            "tickers_data": sorted_tickers,
            "selected_tickers": [],
            "result_data": None,
            "date_str": get_previous_day_str(),
            "default_tickers": default_tickers,
            "error_msg": "티커를 하나 이상 선택해주세요."
        })

    # ✅ 공통 함수 재사용
    date_str = get_previous_day_str()
    result_data, selected_tickers = load_orderbook_data(tickers, date_str)

    return templates.TemplateResponse("index.html", {
        "request": request,
        "tickers_data": sorted_tickers,
        "selected_tickers": selected_tickers,
        "result_data": result_data,
        "date_str": date_str,
        "default_tickers": default_tickers
    })

@app.post("/add_ticker")
async def add_ticker(request: Request, exchange: str = Form(...), ticker: str = Form(...)):
    # Load current tickers and add the new one
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    symbol = ticker.strip().upper()  # normalize input symbol to uppercase
    if symbol and exchange:
        # Ensure the exchange key exists
        if exchange not in tickers_data:
            tickers_data[exchange] = []
        # Avoid duplicates
        if symbol not in tickers_data[exchange]:
            tickers_data[exchange].append(symbol)
    # Save back to JSON
    with open("tickers.json", "w") as f:
        json.dump(tickers_data, f, indent=4)
    # Redirect back to main page
    return RedirectResponse(url="/", status_code=303)

@app.post("/delete_ticker")
async def delete_ticker(request: Request, exchange: str = Form(...), ticker: str = Form(...)):
    # Load current tickers and remove the specified one
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
        
    # 기본 티커는 삭제 불가
    if exchange in default_tickers and ticker in default_tickers[exchange]:
        return RedirectResponse(url="/", status_code=303)
    
    
    if exchange in tickers_data and ticker in tickers_data[exchange]:
        tickers_data[exchange].remove(ticker)
        # If list becomes empty, optionally remove the exchange key
        if not tickers_data[exchange]:
            tickers_data.pop(exchange)
    # Save back to JSON
    with open("tickers.json", "w") as f:
        json.dump(tickers_data, f, indent=4)
    return RedirectResponse(url="/", status_code=303)


In [11]:
def load_orderbook_data(tickers: Optional[List[str]], date_str: str):
    result_data = {}
    selected_by_exchange = {}

    if tickers:
        if len(tickers) > 5:
            tickers = tickers[:5]

        for entry in tickers:
            if ":" in entry:
                ex, symbol = entry.split(":", 1)
                selected_by_exchange.setdefault(ex, []).append(symbol)

        for exchange, symbols in selected_by_exchange.items():
            for ticker_symbol in symbols:
                ticker_key = f"{exchange} - {ticker_symbol}"
                result_data[ticker_key] = []
                pattern = f"orderbook_data/{date_str}/{exchange.lower()}_orderbook_{date_str}_*.json"
                for filename in sorted(glob.glob(pattern)):
                    try:
                        with open(filename, "r", encoding="utf-8") as f:
                            entries = json.load(f)
                        for data in entries:
                            if not isinstance(data, dict):
                                continue
                            ts_ms = int(data.get("timestamp", 0))
                            dt = datetime.fromtimestamp(ts_ms / 1000.0, tz=KST)
                            time_str = dt.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3] + " KST"

                            snapshot = {
                                "timestamp": ts_ms,
                                "time": time_str,
                                "bids": data.get("bids", [])[:10],
                                "asks": data.get("asks", [])[:10]
                            }
                            result_data[ticker_key].append(snapshot)
                    except Exception as e:
                        print(f"❗ Error reading {filename}: {e}")
                        continue

        for ticker_key in result_data:
            result_data[ticker_key].sort(key=lambda x: x["timestamp"])

    return result_data, tickers or []


In [12]:
# ticker 관련 함수들
@app.post("/add_ticker")
async def add_ticker(request: Request, exchange: str = Form(...), ticker: str = Form(...)):
    # Load current tickers and add the new one
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    symbol = ticker.strip().upper()  # normalize input symbol to uppercase
    if symbol and exchange:
        # Ensure the exchange key exists
        if exchange not in tickers_data:
            tickers_data[exchange] = []
        # Avoid duplicates
        if symbol not in tickers_data[exchange]:
            tickers_data[exchange].append(symbol)
    # Save back to JSON
    with open("tickers.json", "w") as f:
        json.dump(tickers_data, f, indent=4)
    # Redirect back to main page
    return RedirectResponse(url="/", status_code=303)

@app.post("/delete_ticker")
async def delete_ticker(request: Request, exchange: str = Form(...), ticker: str = Form(...)):
    # Load current tickers and remove the specified one
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    if exchange in tickers_data and ticker in tickers_data[exchange]:
        tickers_data[exchange].remove(ticker)
        # If list becomes empty, optionally remove the exchange key
        if not tickers_data[exchange]:
            tickers_data.pop(exchange)
    # Save back to JSON
    with open("tickers.json", "w") as f:
        json.dump(tickers_data, f, indent=4)
    return RedirectResponse(url="/", status_code=303)

In [13]:
# 검색 기능 추가
@app.post("/search_log")
async def search_log(
    request: Request,
    log_date: str = Form(...),
    log_exchange: str = Form(""),
    tickers: Optional[List[str]] = Form(None)
):
    log_results = []
    try:
        with open("server.log", "r", encoding="utf-8") as f:
            for line in f:
                if log_date in line and (not log_exchange or log_exchange.lower() in line.lower()):
                    log_results.append(line.strip())
    except FileNotFoundError:
        log_results.append("⚠️ server.log 파일이 없습니다.")

    # load tickers data
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    sorted_tickers = {ex: sorted(tickers_data[ex]) for ex in sorted(tickers_data)}

    # 공통 로직 재사용
    date_str = get_previous_day_str()
    result_data, selected_tickers = load_orderbook_data(tickers, date_str)

    return templates.TemplateResponse("index.html", {
        "request": request,
        "tickers_data": sorted_tickers,
        "selected_tickers": selected_tickers,
        "result_data": result_data or None,
        "date_str": date_str,
        "default_tickers": default_tickers,
        "log_results": log_results,
        "log_date": log_date,
        "log_exchange": log_exchange
    })
    return templates.TemplateResponse("index.html", context)


In [14]:
pip install nest_asyncio


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install uvicorn


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
# jupyter 실행 시 필요
import nest_asyncio
import uvicorn

In [ ]:
# jupyter 실행 
if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app)

INFO:     Started server process [22596]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


📌 Exchange: Upbit Ticker: BTC/KRW
📌 Exchange: Upbit Ticker: BTC/JPN
📌 Exchange: Korbit Ticker: BTC/KRW
📌 Exchange: Korbit Ticker: BTC/JPN
📌 Exchange: Bithumb Ticker: BTC/KRW
📌 Exchange: Coinone Ticker: BTC/KRW
INFO:     127.0.0.1:53914 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53915 - "POST /select HTTP/1.1" 200 OK
INFO:     127.0.0.1:53916 - "POST /search_log HTTP/1.1" 200 OK
INFO:     127.0.0.1:53920 - "POST /select HTTP/1.1" 200 OK
